In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
import folium
import scipy
import json
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 5]
pd.set_option('display.float_format', lambda x:'%f'%x)
from IPython.display import display
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

# EDA

In [39]:
train = pd.read_csv('../data/tmdb/train.csv')
train[:5]

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.000000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.000000,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [40]:
train.shape

(3000, 23)

In [41]:
test = pd.read_csv('../data/tmdb/test.csv')
test[:5]

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,/tnftmLMemPLduW6MRyZE0ZUD19z.jpg,NaN,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.000000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,/9MgBNBqlH1sG4yG2u4XkwI5CoJa.jpg,"[{'name': 'Woolner Brothers Pictures Inc.', 'i...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,/ed6nD7h9sbojSWY2qrnDcSvDFko.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,/sEUG3qjxwHjxkzuO7plrRHhOZUH.jpg,"[{'name': 'TS Productions', 'id': 313}, {'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.000000,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,/n4WC3zbelz6SG7rhkWbf8m9pMHB.jpg,NaN,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [42]:
test.shape

(4398, 22)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9399 entries, 0 to 9398
Columns: 204 entries, budget to production_companies_etc
dtypes: float64(31), int64(173)
memory usage: 14.7 MB


In [44]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 22 columns):
id                       4398 non-null int64
belongs_to_collection    877 non-null object
budget                   4398 non-null int64
genres                   4382 non-null object
homepage                 1420 non-null object
imdb_id                  4398 non-null object
original_language        4398 non-null object
original_title           4398 non-null object
overview                 4384 non-null object
popularity               4398 non-null float64
poster_path              4397 non-null object
production_companies     4140 non-null object
production_countries     4296 non-null object
release_date             4397 non-null object
runtime                  4394 non-null float64
spoken_languages         4356 non-null object
status                   4396 non-null object
tagline                  3535 non-null object
title                    4395 non-null object
Keywords            

In [45]:
df.describe()

,budget,popularity,popularity2,rating,totalVotes,release_month,release_day,release_year,release_dayofweek,release_quarter,weightedRating,originalBudget,inflationBudget,genders_0_crew,genders_1_crew,genders_2_crew,_collection_name,_num_Keywords,_num_cast,_popularity_mean_year,_budget_runtime_ratio,_budget_popularity_ratio,_budget_year_ratio,_releaseYear_popularity_ratio,_releaseYear_popularity_ratio2,_popularity_totalVotes_ratio,_rating_popularity_ratio,_rating_totalVotes_ratio,_totalVotes_releaseYear_ratio,_budget_rating_ratio,_runtime_rating_ratio,_budget_totalVotes_ratio,has_homepage,isbelongs_to_collectionNA,isTaglineNA,isOriginalLanguageEng,isTitleDifferent,isMovieReleased,original_title_letter_count,original_title_word_count,title_word_count,overview_word_count,tagline_word_count,production_countries_count,production_companies_count,cast_count,crew_count,meanruntimeByYear,meanPopularityByYear,meanBudgetByYear,meantotalVotesByYear,meanTotalVotesByRating,medianBudgetByYear,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Australia,Belgium,Brazil,Canada,China,Czech Republic,Denmark,France,Germany,Hong Kong,India,Ireland,Italy,Japan,Luxembourg,Mexico,Netherlands,New Zealand,Russia,South Africa,South Korea,Spain,Sweden,United Kingdom,United States of America,production_countries_etc,Dansk,Deutsch,English,Español,Français,Italiano,Latin,Magyar,Nederlands,Polski,Português,Pусский,Tiếng Việt,Türkçe,spoken_languages_etc,svenska,Český,ελληνικά,עִבְרִית,العربية,فارسی,हिन्दी,தமிழ்,ภาษาไทย,广州话 / 廣州話,日本語,普通话,한국어/조선말,Amblin Entertainment,American Zoetrope,BBC Films,Blumhouse Productions,Canal+,Canal+ España,Cannon Group,Carolco Pictures,Castle Rock Entertainment,Ciné+,Columbia Pictures,Columbia Pictures Corporation,Davis Entertainment,Dimension Films,DreamWorks SKG,Dune Entertainment,Dune Entertainment III,Emmett/Furla Films,Eon Productions,Epsilon Motion Pictures,EuropaCorp,Film4,Focus Features,Fox 2000 Pictures,Fox Searchlight Pictures,France 2 Cinéma,Gaumont,Happy Madison Productions,Hollywood Pictures,Icon Entertainment International,Imagine Entertainment,Ingenious Film Partners,Killer Films,Lakeshore Entertainment,Legendary Pictures,Lions Gate Films,Lionsgate,Malpaso Productions,Metro-Goldwyn-Mayer (MGM),Millennium Films,Miramax Films,Morgan Creek Productions,NPV Entertainment,New Line Cinema,New Regency Pictures,Original Film,Orion Pictures,Paramount Pictures,Participant Media,Phoenix Pictures,PolyGram Filmed Entertainment,Regency Enterprises,Relativity Media,Revolution Studios,Scott Free Productions,Scott Rudin Productions,Screen Gems,Silver Pictures,Spyglass Entertainment,Studio Babelsberg,StudioCanal,Summit Entertainment,TF1 Films Production,TSG Entertainment,The Weinstein Company,Touchstone Pictures,TriStar Pictures,Twentieth Century Fox Film Corporation,UK Film Council,United Artists,Universal Pictures,Village Roadshow Pictures,Walt Disney Pictures,Warner Bros.,Wild Bunch,Wildwood Enterprises,Working Title Films,production_companies_etc
count,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000000,9399.000

In [46]:
test.describe()

,id,budget,popularity,runtime
count,4398.000000,4398.000000,4398.000000,4394.000000
mean,5199.500000,22649291.117326,8.550230,107.622212
std,1269.737571,36899910.881339,12.209014,21.058290
min,3001.000000,0.000000,0.000001,0.000000
25%,4100.250000,0.000000,3.895186,94.000000
50%,5199.500000,7450000.000000,7.482241,104.000000
75%,6298.750000,28000000.000000,10.938524,118.000000
max,7398.000000,260000000.000000,547.488298,320.000000


In [2]:
train.hist(bins=100, figsize=(15,5))

NameError: name 'train' is not defined

In [ ]:
test.hist(bins=100, figsize=(15,5))

In [ ]:
train.describe(include='O')

In [ ]:
test.describe(include='O')

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

## Features
Data Description id - Integer unique id of each movie

belongs_to_collection - Contains the TMDB Id, Name, Movie Poster and Backdrop URL of a movie in JSON format. You can see the Poster and Backdrop Image like this: https://image.tmdb.org/t/p/original/. Example: https://image.tmdb.org/t/p/original//iEhb00TGPucF0b4joM1ieyY026U.jpg

budget:Budget of a movie in dollars. 0 values mean unknown.

genres : Contains all the Genres Name & TMDB Id in JSON Format

homepage - Contains the official homepage URL of a movie. Example: http://sonyclassics.com/whiplash/ , this is the homepage of Whiplash movie.

imdb_id - IMDB id of a movie (string). You can visit the IMDB Page like this: https://www.imdb.com/title/

original_language - Two digit code of the original language, in which the movie was made. Like: en = English, fr = french.

original_title - The original title of a movie. Title & Original title may differ, if the original title is not in English.

overview - Brief description of the movie.

popularity - Popularity of the movie in float.

poster_path - Poster path of a movie. You can see the full image like this: https://image.tmdb.org/t/p/original/

production_companies - All production company name and TMDB id in JSON format of a movie.

production_countries - Two digit code and full name of the production company in JSON format.

release_date - Release date of a movie in mm/dd/yy format.

runtime - Total runtime of a movie in minutes (Integer).

spoken_languages - Two digit code and full name of the spoken language.

status - Is the movie released or rumored?

tagline - Tagline of a movie

title - English title of a movie

Keywords - TMDB Id and name of all the keywords in JSON format.

cast - All cast TMDB id, name, character name, gender (1 = Female, 2 = Male) in JSON format

crew - Name, TMDB id, profile path of various kind of crew members job like Director, Writer, Art, Sound etc.

revenue - Total revenue earned by a movie in dollars.

### Budget vs Revenue

In [ ]:
sns.jointplot(x="budget", y="revenue", data=train, height=11, ratio=4, color="g")
plt.show()

### Popularity vs Revenue

In [ ]:
sns.jointplot(x="popularity", y="revenue", data=train, height=11, ratio=4, color="g")
plt.show()

### Runtime vs Revenue

In [ ]:
sns.jointplot(x="runtime", y="revenue", data=train, height=11, ratio=4, color="g")
plt.show()

### Revenue (Target Variable) Distribution

In [ ]:
sns.distplot(train.revenue)

In [ ]:
train.revenue.describe()

Because revenue variable is skewed, let's calculate log of it.

In [ ]:
train['logRevenue'] = np.log1p(train['revenue'])
sns.distplot(train['logRevenue'] )

### Release Date: Get Month, Day Year, Week day, quarter from release date

In [ ]:
#Since only last two digits of year are provided, this is the correct way of getting the year.
train[['release_month','release_day','release_year']]=train['release_date'].str.split('/',expand=True).replace(np.nan, -1).astype(int)
# Some rows have 4 digits of year instead of 2, that's why I am applying (train['release_year'] < 100) this condition
train.loc[ (train['release_year'] <= 19) & (train['release_year'] < 100), "release_year"] += 2000
train.loc[ (train['release_year'] > 19)  & (train['release_year'] < 100), "release_year"] += 1900

releaseDate = pd.to_datetime(train['release_date']) 
train['release_dayofweek'] = releaseDate.dt.dayofweek
train['release_quarter'] = releaseDate.dt.quarter

### Release Year Count

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_year'].sort_values())
plt.title("Movie Release count by Year",fontsize=20)
loc, labels = plt.xticks()
plt.xticks(fontsize=12,rotation=90)
plt.show()

### Release month count

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_month'].sort_values())
plt.title("Release Month Count",fontsize=20)
loc, labels = plt.xticks()
loc, labels = loc, ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
plt.xticks(loc, labels,fontsize=20)
plt.show()

### Release day count

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_day'].sort_values())
plt.title("Release Day Count",fontsize=20)
plt.xticks(fontsize=20)
plt.show()

### Release Day of Week

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_dayofweek'].sort_values())
plt.title("Total movies released on Day Of Week",fontsize=20)
loc, labels = plt.xticks()
loc, labels = loc, ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
plt.xticks(loc, labels,fontsize=20)
plt.show()

### Release quarter Count

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_quarter'].sort_values())
plt.title("Total movies released in a quarter",fontsize=20)
plt.show()

### Release Year vs Revenue

In [ ]:
train['meanRevenueByYear'] = train.groupby("release_year")["revenue"].aggregate('mean')
train['meanRevenueByYear'].plot(figsize=(15,10),color="g")
plt.xticks(np.arange(1920,2018,4))
plt.xlabel("Release Year")
plt.ylabel("Revenue")
plt.title("Movie Mean Revenue By Year",fontsize=20)
plt.show()

### Release Month Vs Revenue Plot

In [ ]:
train['meanRevenueByMonth'] = train.groupby("release_month")["revenue"].aggregate('mean')
train['meanRevenueByMonth'].plot(figsize=(15,10),color="g")
plt.xlabel("Release Month")
plt.ylabel("Revenue")
plt.title("Movie Mean Revenue Release Month",fontsize=20)
plt.show()

### Release day of week vs Revenue

In [ ]:
train['meanRevenueByDayOfWeek'] = train.groupby("release_dayofweek")["revenue"].aggregate('mean')
train['meanRevenueByDayOfWeek'].plot(figsize=(15,10),color="g")
plt.xlabel("Day of Week")
plt.ylabel("Revenue")
plt.title("Movie Mean Revenue by Day of Week",fontsize=20)
plt.show()

In [ ]:
train['meanRevenueByQuarter'] = train.groupby("release_quarter")["revenue"].aggregate('mean')
train['meanRevenueByQuarter'].plot(figsize=(15,10),color="g")
plt.xticks(np.arange(1,5,1))
plt.xlabel("Quarter")
plt.ylabel("Revenue")
plt.title("Movie Mean Revenue by Quarter",fontsize=20)
plt.show()

In [36]:
df = pd.read_csv('../data/tmdb/all_data.csv')
df[:5]

,Unnamed: 0,budget,popularity,popularity2,rating,totalVotes,release_month,release_day,release_year,release_dayofweek,release_quarter,weightedRating,originalBudget,inflationBudget,genders_0_crew,genders_1_crew,genders_2_crew,_collection_name,_num_Keywords,_num_cast,_popularity_mean_year,_budget_runtime_ratio,_budget_popularity_ratio,_budget_year_ratio,_releaseYear_popularity_ratio,_releaseYear_popularity_ratio2,_popularity_totalVotes_ratio,_rating_popularity_ratio,_rating_totalVotes_ratio,_totalVotes_releaseYear_ratio,_budget_rating_ratio,_runtime_rating_ratio,_budget_totalVotes_ratio,has_homepage,isbelongs_to_collectionNA,isTaglineNA,isOriginalLanguageEng,isTitleDifferent,isMovieReleased,original_title_letter_count,original_title_word_count,title_word_count,overview_word_count,tagline_word_count,production_countries_count,production_companies_count,cast_count,crew_count,meanruntimeByYear,meanPopularityByYear,meanBudgetByYear,meantotalVotesByYear,meanTotalVotesByRating,medianBudgetByYear,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Australia,Belgium,Brazil,Canada,China,Czech Republic,Denmark,France,Germany,Hong Kong,India,Ireland,Italy,Japan,Luxembourg,Mexico,Netherlands,New Zealand,Russia,South Africa,South Korea,Spain,Sweden,United Kingdom,United States of America,production_countries_etc,Dansk,Deutsch,English,Español,Français,Italiano,Latin,Magyar,Nederlands,Polski,Português,Pусский,Tiếng Việt,Türkçe,spoken_languages_etc,svenska,Český,ελληνικά,עִבְרִית,العربية,فارسی,हिन्दी,தமிழ்,ภาษาไทย,广州话 / 廣州話,日本語,普通话,한국어/조선말,Amblin Entertainment,American Zoetrope,BBC Films,Blumhouse Productions,Canal+,Canal+ España,Cannon Group,Carolco Pictures,Castle Rock Entertainment,Ciné+,Columbia Pictures,Columbia Pictures Corporation,Davis Entertainment,Dimension Films,DreamWorks SKG,Dune Entertainment,Dune Entertainment III,Emmett/Furla Films,Eon Productions,Epsilon Motion Pictures,EuropaCorp,Film4,Focus Features,Fox 2000 Pictures,Fox Searchlight Pictures,France 2 Cinéma,Gaumont,Happy Madison Productions,Hollywood Pictures,Icon Entertainment International,Imagine Entertainment,Ingenious Film Partners,Killer Films,Lakeshore Entertainment,Legendary Pictures,Lions Gate Films,Lionsgate,Malpaso Productions,Metro-Goldwyn-Mayer (MGM),Millennium Films,Miramax Films,Morgan Creek Productions,NPV Entertainment,New Line Cinema,New Regency Pictures,Original Film,Orion Pictures,Paramount Pictures,Participant Media,Phoenix Pictures,PolyGram Filmed Entertainment,Regency Enterprises,Relativity Media,Revolution Studios,Scott Free Productions,Scott Rudin Productions,Screen Gems,Silver Pictures,Spyglass Entertainment,Studio Babelsberg,StudioCanal,Summit Entertainment,TF1 Films Production,TSG Entertainment,The Weinstein Company,Touchstone Pictures,TriStar Pictures,Twentieth Century Fox Film Corporation,UK Film Council,United Artists,Universal Pictures,Village Roadshow Pictures,Walt Disney Pictures,Warner Bros.,Wild Bunch,Wildwood Enterprises,Working Title Films,production_companies_etc
0,0,16.454568,6.575393,10.400000,5.000000,482.000000,2,20,2015,4.000000,1.000000,5.922402,14000000,14756000.000000,59,0,13,282,4,24,0.713006,0.176931,2.502446,0.000004,306.445562,0.003263,73.303603,0.760411,96.400000,0.239206,3.290914,18.600000,0.034138,0,0,0,1,0,1,22,5,5.000000,29.000000,11.000000,1,3,24,72,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,17.504390,8.248895,15.229000,6.400000,1528.000000,8,6,2004,4.000000,3.000000,6.386946,40000000,50080000.000000,1,4,4,641,4,20,1.104025,0.154906,2.122028,0.000004,242.941630,0.004116,185.236932,0.775861,238.750000,0.762475

In [37]:
df.rename(columns={'Unnamed: 0':''},inplace=True)
df[:5]

,,budget,popularity,popularity2,rating,totalVotes,release_month,release_day,release_year,release_dayofweek,release_quarter,weightedRating,originalBudget,inflationBudget,genders_0_crew,genders_1_crew,genders_2_crew,_collection_name,_num_Keywords,_num_cast,_popularity_mean_year,_budget_runtime_ratio,_budget_popularity_ratio,_budget_year_ratio,_releaseYear_popularity_ratio,_releaseYear_popularity_ratio2,_popularity_totalVotes_ratio,_rating_popularity_ratio,_rating_totalVotes_ratio,_totalVotes_releaseYear_ratio,_budget_rating_ratio,_runtime_rating_ratio,_budget_totalVotes_ratio,has_homepage,isbelongs_to_collectionNA,isTaglineNA,isOriginalLanguageEng,isTitleDifferent,isMovieReleased,original_title_letter_count,original_title_word_count,title_word_count,overview_word_count,tagline_word_count,production_countries_count,production_companies_count,cast_count,crew_count,meanruntimeByYear,meanPopularityByYear,meanBudgetByYear,meantotalVotesByYear,meanTotalVotesByRating,medianBudgetByYear,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Australia,Belgium,Brazil,Canada,China,Czech Republic,Denmark,France,Germany,Hong Kong,India,Ireland,Italy,Japan,Luxembourg,Mexico,Netherlands,New Zealand,Russia,South Africa,South Korea,Spain,Sweden,United Kingdom,United States of America,production_countries_etc,Dansk,Deutsch,English,Español,Français,Italiano,Latin,Magyar,Nederlands,Polski,Português,Pусский,Tiếng Việt,Türkçe,spoken_languages_etc,svenska,Český,ελληνικά,עִבְרִית,العربية,فارسی,हिन्दी,தமிழ்,ภาษาไทย,广州话 / 廣州話,日本語,普通话,한국어/조선말,Amblin Entertainment,American Zoetrope,BBC Films,Blumhouse Productions,Canal+,Canal+ España,Cannon Group,Carolco Pictures,Castle Rock Entertainment,Ciné+,Columbia Pictures,Columbia Pictures Corporation,Davis Entertainment,Dimension Films,DreamWorks SKG,Dune Entertainment,Dune Entertainment III,Emmett/Furla Films,Eon Productions,Epsilon Motion Pictures,EuropaCorp,Film4,Focus Features,Fox 2000 Pictures,Fox Searchlight Pictures,France 2 Cinéma,Gaumont,Happy Madison Productions,Hollywood Pictures,Icon Entertainment International,Imagine Entertainment,Ingenious Film Partners,Killer Films,Lakeshore Entertainment,Legendary Pictures,Lions Gate Films,Lionsgate,Malpaso Productions,Metro-Goldwyn-Mayer (MGM),Millennium Films,Miramax Films,Morgan Creek Productions,NPV Entertainment,New Line Cinema,New Regency Pictures,Original Film,Orion Pictures,Paramount Pictures,Participant Media,Phoenix Pictures,PolyGram Filmed Entertainment,Regency Enterprises,Relativity Media,Revolution Studios,Scott Free Productions,Scott Rudin Productions,Screen Gems,Silver Pictures,Spyglass Entertainment,Studio Babelsberg,StudioCanal,Summit Entertainment,TF1 Films Production,TSG Entertainment,The Weinstein Company,Touchstone Pictures,TriStar Pictures,Twentieth Century Fox Film Corporation,UK Film Council,United Artists,Universal Pictures,Village Roadshow Pictures,Walt Disney Pictures,Warner Bros.,Wild Bunch,Wildwood Enterprises,Working Title Films,production_companies_etc
0,0,16.454568,6.575393,10.400000,5.000000,482.000000,2,20,2015,4.000000,1.000000,5.922402,14000000,14756000.000000,59,0,13,282,4,24,0.713006,0.176931,2.502446,0.000004,306.445562,0.003263,73.303603,0.760411,96.400000,0.239206,3.290914,18.600000,0.034138,0,0,0,1,0,1,22,5,5.000000,29.000000,11.000000,1,3,24,72,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,17.504390,8.248895,15.229000,6.400000,1528.000000,8,6,2004,4.000000,3.000000,6.386946,40000000,50080000.000000,1,4,4,641,4,20,1.104025,0.154906,2.122028,0.000004,242.941630,0.004116,185.236932,0.775861,238.750000,0.762475,2.735061,

In [38]:
df.set_index('',inplace=True)
df[:5]

,budget,popularity,popularity2,rating,totalVotes,release_month,release_day,release_year,release_dayofweek,release_quarter,weightedRating,originalBudget,inflationBudget,genders_0_crew,genders_1_crew,genders_2_crew,_collection_name,_num_Keywords,_num_cast,_popularity_mean_year,_budget_runtime_ratio,_budget_popularity_ratio,_budget_year_ratio,_releaseYear_popularity_ratio,_releaseYear_popularity_ratio2,_popularity_totalVotes_ratio,_rating_popularity_ratio,_rating_totalVotes_ratio,_totalVotes_releaseYear_ratio,_budget_rating_ratio,_runtime_rating_ratio,_budget_totalVotes_ratio,has_homepage,isbelongs_to_collectionNA,isTaglineNA,isOriginalLanguageEng,isTitleDifferent,isMovieReleased,original_title_letter_count,original_title_word_count,title_word_count,overview_word_count,tagline_word_count,production_countries_count,production_companies_count,cast_count,crew_count,meanruntimeByYear,meanPopularityByYear,meanBudgetByYear,meantotalVotesByYear,meanTotalVotesByRating,medianBudgetByYear,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Australia,Belgium,Brazil,Canada,China,Czech Republic,Denmark,France,Germany,Hong Kong,India,Ireland,Italy,Japan,Luxembourg,Mexico,Netherlands,New Zealand,Russia,South Africa,South Korea,Spain,Sweden,United Kingdom,United States of America,production_countries_etc,Dansk,Deutsch,English,Español,Français,Italiano,Latin,Magyar,Nederlands,Polski,Português,Pусский,Tiếng Việt,Türkçe,spoken_languages_etc,svenska,Český,ελληνικά,עִבְרִית,العربية,فارسی,हिन्दी,தமிழ்,ภาษาไทย,广州话 / 廣州話,日本語,普通话,한국어/조선말,Amblin Entertainment,American Zoetrope,BBC Films,Blumhouse Productions,Canal+,Canal+ España,Cannon Group,Carolco Pictures,Castle Rock Entertainment,Ciné+,Columbia Pictures,Columbia Pictures Corporation,Davis Entertainment,Dimension Films,DreamWorks SKG,Dune Entertainment,Dune Entertainment III,Emmett/Furla Films,Eon Productions,Epsilon Motion Pictures,EuropaCorp,Film4,Focus Features,Fox 2000 Pictures,Fox Searchlight Pictures,France 2 Cinéma,Gaumont,Happy Madison Productions,Hollywood Pictures,Icon Entertainment International,Imagine Entertainment,Ingenious Film Partners,Killer Films,Lakeshore Entertainment,Legendary Pictures,Lions Gate Films,Lionsgate,Malpaso Productions,Metro-Goldwyn-Mayer (MGM),Millennium Films,Miramax Films,Morgan Creek Productions,NPV Entertainment,New Line Cinema,New Regency Pictures,Original Film,Orion Pictures,Paramount Pictures,Participant Media,Phoenix Pictures,PolyGram Filmed Entertainment,Regency Enterprises,Relativity Media,Revolution Studios,Scott Free Productions,Scott Rudin Productions,Screen Gems,Silver Pictures,Spyglass Entertainment,Studio Babelsberg,StudioCanal,Summit Entertainment,TF1 Films Production,TSG Entertainment,The Weinstein Company,Touchstone Pictures,TriStar Pictures,Twentieth Century Fox Film Corporation,UK Film Council,United Artists,Universal Pictures,Village Roadshow Pictures,Walt Disney Pictures,Warner Bros.,Wild Bunch,Wildwood Enterprises,Working Title Films,production_companies_etc
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,16.454568,6.575393,10.400000,5.000000,482.000000,2,20,2015,4.000000,1.000000,5.922402,14000000,14756000.000000,59,0,13,282,4,24,0.713006,0.176931,2.502446,0.000004,306.445562,0.003263,73.303603,0.760411,96.400000,0.239206,3.290914,18.600000,0.034138,0,0,0,1,0,1,22,5,5.000000,29.000000,11.000000,1,3,24,72,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,17.504390,8.248895,15.229000,6.